In [ ]:
#required libraries
%matplotlib qt
import numpy as np
import hyperspy.api as hp
from ncempy.io import dm
import matplotlib.pyplot as plt

In [ ]:
#load spectrum image
dataset = dm.dmReader("D:\Documents\Summer 2020 Research\Sample Data\SI_2.dm4")
psf = dm.dmReader("D:\Documents\Summer 2020 Research\Sample Data\psf.dm4")

#index for separate spectra of the same size
Spec = dataset['data'][:1778,0,0]
PSF = psf['data'][:,0,0]

dataset.keys()

In [ ]:
#Functions

#Normalize spectrum based on total electron counts
def normalizeS(S):
	S_norm = S/np.sum(S)
	return S_norm

#Maximum-Entropy algorithm
def ISRA(iterations, PSF_norm, Spec):
    ISRA5 = np.copy(Spec)
    for i in range(iterations):
        ISRA1 = np.convolve(PSF_norm, Spec, 'same')
        ISRA2 = np.convolve(PSF_norm, ISRA5, 'same')
        ISRA3 = np.convolve(PSF_norm, ISRA2, 'same')
        ISRA4 = ISRA1/ISRA3
        ISRA5 *= ISRA4
    return ISRA5

#Richardson-Lucy algorithm (from Edson Bellido)
def RL(iterations, PSF_norm, Spec):
    RL4 = np.copy(Spec)
    for ii in range(iterations):
        RL1 = np.convolve(PSF_norm, RL4, 'same')
        RL2 = Spec/RL1
        RL3 = np.convolve(PSF_norm, RL2, 'same')
        RL4 *= RL3
    return RL4

In [ ]:
#normalize the PSF
PSF_norm = normalizeS(PSF)

In [ ]:
#perform deconvolution
ISRA = ISRA(20, PSF_norm, Spec)
RL = RL(20, PSF_norm, Spec)

In [ ]:
#plot

fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax1.set_title("Spec")
ax2.set_title("PSF")
ax1.plot(Spec)
ax2.plot(PSF)

fig_1 = plt.figure()
ax_test = fig_1.add_subplot(111)
ax_test.plot(abs(ISRA))
fig_1.suptitle("ISRA")

fig_2 = plt.figure()
ax_test = fig_2.add_subplot(111)
ax_test.plot(abs(RL))
fig_2.suptitle("RL")

plt.show()